# TensorFlow and TensorNetwork Devices

This notebook compares the performance of the TensorNetwork device used with TensorFlow.
We find the speedup over autograd on default qubit to be significant and allows a new tier of networks to be optimized with relative efficieny.
Using the Tensor Network backend we are able to optimize a 20-qubit network in roughly 5 minutes on a laptop computer.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import time

import tensorflow as tf

from context import QNetOptimizer as QNopt

### Setup

In [2]:
def bell_state_local_RY(settings, wires):
    qml.Hadamard(wires[0])
    qml.CNOT(wires=wires[0:2])
        
    qml.RY(settings[0], wires=wires[0])
    qml.RY(settings[1], wires=wires[1])

def static_nlocal_prepare_nodes(n):
    return [QNopt.PrepareNode(1, [2*i, 2*i + 1], bell_state_local_RY, 2) for i in range(n)]

def local_RY_measure_nodes(n):
    meas_nodes = [QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1)]
    meas_nodes.extend([
        QNopt.MeasureNode(2, 2, [2*i + 1, 2*i + 2], QNopt.local_RY, 2) for i in range(0,n-1)
    ])
    meas_nodes.append(QNopt.MeasureNode(2, 2, [2*n-1], QNopt.local_RY, 1))
    
    return meas_nodes

def tn_optimize_nlocal_chain(n):
    print("n : ", n)

    tn_prep_nodes = static_nlocal_prepare_nodes(n)
    tn_meas_nodes = local_RY_measure_nodes(n)
    
    tn_nlocal_ansatz = QNopt.NetworkAnsatz(tn_prep_nodes, tn_meas_nodes)
    tn_nlocal_ansatz.dev = qml.device("default.tensor.tf",
        wires=tn_nlocal_ansatz.network_wires,
        representation="exact",
        contraction_method="auto"
    )
    
    np_settings = tn_nlocal_ansatz.rand_scenario_settings()

    tn_settings = [
        [tf.Variable(settings) for settings in np_settings[0]],
        [tf.Variable(settings) for settings in np_settings[1]],
    ]

    tn_cost = QNopt.nlocal_chain_cost_22(tn_nlocal_ansatz, interface="tf", diff_method="backprop")
    tn_opt_dict = QNopt.gradient_descent(
        tn_cost,
        tn_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 3,
        interface="tf"
    )
    
    print("max score : ", tn_opt_dict["opt_score"])
    
    return tn_opt_dict  

def tf_optimize_nlocal_chain(n):
    print("n : ", n)

    tf_prep_nodes = static_nlocal_prepare_nodes(n)
    tf_meas_nodes = local_RY_measure_nodes(n)
    
    tf_nlocal_ansatz = QNopt.NetworkAnsatz(tf_prep_nodes, tf_meas_nodes)

    np_settings = tf_nlocal_ansatz.rand_scenario_settings()

    tf_settings = [
        [tf.Variable(settings) for settings in np_settings[0]],
        [tf.Variable(settings) for settings in np_settings[1]],
    ]

    tf_cost = QNopt.nlocal_chain_cost_22(tf_nlocal_ansatz, interface="tf", diff_method="backprop")
    tf_opt_dict = QNopt.gradient_descent(
        tf_cost,
        tf_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 3,
        interface="tf"
    )
    
    print("max score : ", tf_opt_dict["opt_score"])
    
    return tf_opt_dict 

def ag_optimize_nlocal_chain(n):
    print("n : ", n)

    ag_prep_nodes = static_nlocal_prepare_nodes(n)
    ag_meas_nodes = local_RY_measure_nodes(n)
    
    ag_nlocal_ansatz = QNopt.NetworkAnsatz(ag_prep_nodes, ag_meas_nodes)
    
    ag_settings = ag_nlocal_ansatz.rand_scenario_settings()

    ag_cost = QNopt.nlocal_chain_cost_22(ag_nlocal_ansatz, interface="autograd", diff_method="backprop")
    ag_opt_dict = QNopt.gradient_descent(
        ag_cost,
        ag_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 3
    )
    
    print("max score : ", ag_opt_dict["opt_score"])
    
    return ag_opt_dict 

## Interface Comparison on Backpropagation

### Autograd

In [3]:
%%time

np.random.seed(1)
ag6_opt_dict = ag_optimize_nlocal_chain(6)

n :  6
iteration :  0 , score :  0.2910119273912153
iteration :  3 , score :  1.170438972976621
iteration :  6 , score :  1.3713732007878927
iteration :  9 , score :  1.4082694211302902
iteration :  12 , score :  1.4134015966515074
max score :  1.4141033563290029
CPU times: user 4.64 s, sys: 167 ms, total: 4.8 s
Wall time: 4.86 s


### TensorFlow

In [4]:
%%time

np.random.seed(1)
tf6_opt_dict = tf_optimize_nlocal_chain(6)

n :  6


2021-09-29 17:17:01.887528: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


iteration :  0 , score :  tf.Tensor(0.2910119273912166, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(1.1704389666959336, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.371373202152422, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.408269421524657, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.4134015967330096, shape=(), dtype=float64)
max score :  tf.Tensor(1.414103356343892, shape=(), dtype=float64)
CPU times: user 26.1 s, sys: 2.12 s, total: 28.3 s
Wall time: 15.8 s


### TensorFlow with TensorNetwork Device

In [5]:
%%time

np.random.seed(1)
tn6_opt_dict = tn_optimize_nlocal_chain(6)

n :  6
iteration :  0 , score :  tf.Tensor(0.2910119273912156, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(1.1704389666959438, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.3713732021524205, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.4082694215246558, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.413401596733006, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141033563438952, shape=(), dtype=float64)
CPU times: user 15.7 s, sys: 552 ms, total: 16.3 s
Wall time: 13.6 s


## Scaling TensorFlow on TensorNetwork Devices

In [6]:
%%time

np.random.seed(1)
n7_opt_dict = tn_optimize_nlocal_chain(7)

n :  7
iteration :  0 , score :  tf.Tensor(0.22747569616624702, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(1.291311561594983, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.4027941585015897, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.4127345535776898, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.4140139673514156, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141865830313487, shape=(), dtype=float64)
CPU times: user 28.4 s, sys: 946 ms, total: 29.4 s
Wall time: 16.9 s


In [7]:
%%time

np.random.seed(1)
n8_opt_dict = tn_optimize_nlocal_chain(8)

n :  8
iteration :  0 , score :  tf.Tensor(0.09333571468401751, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(1.0352364320233645, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.3867815281096727, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.4114182484350573, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.413839586621107, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141627006090363, shape=(), dtype=float64)
CPU times: user 1min 27s, sys: 2.51 s, total: 1min 29s
Wall time: 25.7 s


In [8]:
%%time

np.random.seed(3)
n9_opt_dict = tn_optimize_nlocal_chain(9)

n :  9
iteration :  0 , score :  tf.Tensor(0.06313287953314659, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(1.0021867112636502, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.3889188065625029, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.4121259750140072, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.4139399330166496, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141764073218086, shape=(), dtype=float64)
CPU times: user 7min 36s, sys: 25.9 s, total: 8min 2s
Wall time: 1min 17s


## TensorNetwork Outperforms Default Qubit

In [9]:
%%time

np.random.seed(3)
n9_ag_opt_dict = ag_optimize_nlocal_chain(9)

n :  9
iteration :  0 , score :  0.06313287953314589
iteration :  3 , score :  1.002186701610961
iteration :  6 , score :  1.3889188043775538
iteration :  9 , score :  1.412125974825977
iteration :  12 , score :  1.4139399329842919
max score :  1.414176407316137
CPU times: user 3min 48s, sys: 34.8 s, total: 4min 23s
Wall time: 2min 3s


## Max "Efficient" (<10min) Optimization, $n=10$ (20 Qubits)

In [10]:
%%time

np.random.seed(1)
n10_opt_dict = tn_optimize_nlocal_chain(10)

n :  10
iteration :  0 , score :  tf.Tensor(0.04566642166979497, shape=(), dtype=float64)
iteration :  3 , score :  tf.Tensor(0.5583340198239084, shape=(), dtype=float64)
iteration :  6 , score :  tf.Tensor(1.18085092014525, shape=(), dtype=float64)
iteration :  9 , score :  tf.Tensor(1.3678963583309893, shape=(), dtype=float64)
iteration :  12 , score :  tf.Tensor(1.4075520618921875, shape=(), dtype=float64)
max score :  tf.Tensor(1.4132963148318258, shape=(), dtype=float64)
CPU times: user 38min 19s, sys: 1min 3s, total: 39min 23s
Wall time: 4min 46s
